In [2]:
import numpy as np
%load_ext autoreload
%autoreload 2

from custom_lib import quantum_circuit_ctx_generator, utils, verilog, fixed_point_handler

import qiskit

num_qubits = 3
qc = qiskit.QuantumCircuit(num_qubits)
# qc.rx(1.37, 4)
# qc.rx(2.68, 3)
# qc.rx(5.81, 2)
qc.rx(1.15, 1)
# qc.rx(2.23, 0)
# qc.rz(4.4, 4)
# qc.rz(1.32, 3)
# qc.rz(0.0575, 2)
# qc.rz(4.62, 1)
# qc.rz(2.93, 0)
# qc.cx(0, 1)
# qc.cx(1, 2)
# qc.cx(2, 3)
# qc.cx(3, 4)
from qimax import converter, circuit, splitter


qcs = splitter.qc_to_qcs(qc)
gatess = converter.qcs_to_gatess(qcs)
gatess_with_I = converter.gatess_to_gatess_with_I(gatess, num_qubits)

# for u in gatess_with_I:
#     print(u)

# # CONVERT QISKIT CIRCUIT TO STR
# strs = converter.gatess_to_str(gatess_with_I) 
# with open('test_output.txt', 'w') as f:
#     for item in strs:
#         f.write("%s\n" % item)
#         print(item)



# qc.draw(output='mpl')
print(qiskit.quantum_info.Statevector.from_instruction(qc).data)

import numpy as np
import sys
# sys.path.append('..')
from custom_lib import quantum_circuit_ctx_generator, utils, verilog, fixed_point_handler

res = np.array(qiskit.quantum_info.Statevector.from_instruction(qc).data)

bit_width = 32
fraction_width = 30
hex_res = []
for idx in range(len(res)):
    # line = line.strip("()")     
    # line = line.strip("")
    compl_num = complex(res[idx])

    real_part = compl_num.real
    imaginary_part = compl_num.imag

    # print("Complex Number:", real_part + 1j*imaginary_part)

    real_part_binary = fixed_point_handler.convert_fixedpoint_to_binary(number=real_part, bit_width=bit_width, dot_position=fraction_width)
    imaginary_part_binary = fixed_point_handler.convert_fixedpoint_to_binary(number=imaginary_part, bit_width=bit_width, dot_position=fraction_width)

    real_part_hex = fixed_point_handler.binary_to_hex(binary_str=real_part_binary, bit_width=bit_width)
    imaginary_part_hex = fixed_point_handler.binary_to_hex(binary_str=imaginary_part_binary, bit_width=bit_width)

    print("Index:", idx, " - Hex Form:", real_part_hex[2:]+imaginary_part_hex[2:])

    hex_res.append(real_part_hex[2:]+imaginary_part_hex[2:])

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
[0.8391923+0.j         0.       +0.j         0.       -0.54383479j
 0.       +0.j         0.       +0.j         0.       +0.j
 0.       +0.j         0.       +0.j        ]
Index: 0  - Hex Form: 35b553a100000000
Index: 1  - Hex Form: 0000000000000000
Index: 2  - Hex Form: 00000000dd31cf90
Index: 3  - Hex Form: 0000000000000000
Index: 4  - Hex Form: 0000000000000000
Index: 5  - Hex Form: 0000000000000000
Index: 6  - Hex Form: 0000000000000000
Index: 7  - Hex Form: 0000000000000000


In [ ]:
import numpy as np

# np.set_printoptions(threshold=np.inf, linewidth=np.inf)

def tensor_product(matrix_a, matrix_b):
    """
    Compute the tensor product (Kronecker product) of two matrices.

    Parameters:
    - matrix_a: The first matrix (numpy array).
    - matrix_b: The second matrix (numpy array).

    Returns:
    - A numpy array representing the tensor product of the two matrices.
    """
    return np.kron(matrix_a, matrix_b)

def cx_matrix_normal(n_qubit, ctrl_pos, target_pos):
  M = np.array([[1, 0], [0, 0]])
  P = np.array([[0, 0], [0, 1]])
  X = np.array([[0, 1], [1, 0]])
  I = np.array([[1, 0], [0, 1]])

  left_side_gate = np.array([])
  right_side_gate = np.array([])

  for i in range(n_qubit):
    if(i != ctrl_pos):
      left_side_gate = np.append(left_side_gate, np.array(["I"]))

      if(i != target_pos):
        right_side_gate = np.append(right_side_gate, np.array(["I"]))
      else:
        right_side_gate = np.append(right_side_gate, np.array(["X"]))
    else:
      left_side_gate = np.append(left_side_gate, np.array(["M"]))
      right_side_gate = np.append(right_side_gate, np.array(["P"]))

#   print(left_side_gate)
#   print(right_side_gate)

  left_side = np.array([])
  right_side = np.array([])

  for i in range(n_qubit):
    if(i == 0):
      if(left_side_gate[i] == "I"):
        left_side = I
      elif (left_side_gate[i] == "M"):
        left_side = M

      if(right_side_gate[i] == "I"):
        right_side = I
      elif (right_side_gate[i] == "P"):
        right_side = P
      elif (right_side_gate[i] == "X"):
        right_side = X
    else:
      if(left_side_gate[i] == "I"):
        left_side = tensor_product(left_side, I)
      elif (left_side_gate[i] == "M"):
        left_side = tensor_product(left_side, M)

      if(right_side_gate[i] == "I"):
        right_side = tensor_product(right_side, I)
      elif (right_side_gate[i] == "P"):
        right_side = tensor_product(right_side, P)
      elif (right_side_gate[i] == "X"):
        right_side = tensor_product(right_side, X)

  return left_side + right_side

def cx_matrix_formula(n, control, target):
    """
    Generate the matrix representation of a CX gate for an n-qubit system.

    Args:
        n (int): Number of qubits.
        control (int): Position of the control qubit (0-based index).
        target (int): Position of the target qubit (0-based index).

    Returns:
        np.ndarray: The 2^n x 2^n matrix representation of the CX gate.
    """
    # Size of the matrix
    size = 2**n
    # Initialize the matrix as a zero matrix
    cx_matrix = np.zeros((size, size), dtype=int)

    # Iterate over all row indices
    for i in range(size):
        # Convert row index to binary representation
        binary_i = format(i, f'0{n}b')
        # Check if the control qubit is |1⟩
        if binary_i[control] == '1':
            # Flip the target qubit to get the column index
            binary_j = binary_i[:target] + ('1' if binary_i[target] == '0' else '0') + binary_i[target+1:]

            print(binary_i)
            print(binary_i[:target])
            print(('1' if binary_i[target] == '0' else '0'))
            print(binary_i[target+1:])
            print(binary_i[:target] + ('1' if binary_i[target] == '0' else '0') + binary_i[target+1:])
            j = int(binary_j, 2)
            # Place a 1 at position (i, j)
            cx_matrix[i, j] = 1

            print(i, j)
        else:
            # If control qubit is |0⟩, the target qubit remains unchanged
            cx_matrix[i, i] = 1

    return cx_matrix

In [36]:
cx_matrix_normal(n_qubit=5, ctrl_pos=4, target_pos=3)

array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [60]:
cx_matrix_formula(n=5, control=4, target=3)

00001
000
1
1
00011
1 3
00011
000
0
1
00001
3 1
00101
001
1
1
00111
5 7
00111
001
0
1
00101
7 5
01001
010
1
1
01011
9 11
01011
010
0
1
01001
11 9
01101
011
1
1
01111
13 15
01111
011
0
1
01101
15 13
10001
100
1
1
10011
17 19
10011
100
0
1
10001
19 17
10101
101
1
1
10111
21 23
10111
101
0
1
10101
23 21
11001
110
1
1
11011
25 27
11011
110
0
1
11001
27 25
11101
111
1
1
11111
29 31
11111
111
0
1
11101
31 29


array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [40]:
data_arr = np.array([0, 1, 2, 3, 4, 5])

print(data_arr[:2])
print(data_arr[1+1:])

[0 1]
[2 3 4 5]


In [54]:
import numpy as np
%load_ext autoreload
%autoreload 2

import qiskit
import qiskit.quantum_info

num_qubits = 3
qc = qiskit.QuantumCircuit(num_qubits)
qc.h(0)
print(qiskit.quantum_info.Statevector.from_instruction(qc).data)
from qimax import converter, circuit, splitter


qcs = splitter.qc_to_qcs(qc)
gatess = converter.qcs_to_gatess(qcs)
gatess_with_I = converter.gatess_to_gatess_with_I(gatess, num_qubits)

for u in gatess_with_I:
    print(u)

# CONVERT QISKIT CIRCUIT TO STR
strs = converter.gatess_to_str(gatess_with_I) 
with open('output2.txt', 'w') as f:
    for item in strs:
        f.write("%s\n" % item)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
[0.70710678+0.j 0.70710678+0.j 0.        +0.j 0.        +0.j 0.        +0.j 0.        +0.j 0.        +0.j 0.        +0.j]
[('H', -999, [0]), ['I', -999, [1]], ['I', -999, [2]]]
